In [6]:
import glob
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

import certifi
import ssl
import geopy.geocoders
from geopy.geocoders import Nominatim 
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx

import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium # map rendering library

In [7]:
venues_grouped = pd.read_csv('venues_grouped.csv', index_col=0)
suburbs_venues_sorted = pd.read_csv('venues_sorted.csv', index_col=0)
venues_grouped.head()

,Address,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Astrologer,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Badminton Court,Bagel Shop,Bakery,Bangladeshi Restaurant,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Bay,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Bengali Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buddhist Temple,Buffet,Building,Bulgarian Restaurant,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Casino,Castle,Caucasian Restaurant,Cemetery,Cha Chaan Teng,Chaat Place,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Basketball Court,College Bookstore,College Cafeteria,College Football Field,College Gym,College Rec Center,College Residence Hall,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Confucian Temple,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cricket Ground,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Disc Golf,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donburi Restaurant,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Emergency Room,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Forest,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Golf Course,Golf Driving Range,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Guizhou Restaurant,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hainan Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hubei Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Insurance

In [8]:
# Cluster Suburbs

# set number of clusters
kclusters = 5

venues_grouped_clustering = venues_grouped.drop('Address', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, ..., 0, 3, 0], dtype=int32)

In [9]:
# dataframe that includes the cluster as well as the top 10 venues for each Suburb

# add clustering labels
suburbs_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# merge venues_grouped with venues data to add latitude/longitude for each Suburb
venues_merged = suburbs_venues_sorted.join(df_venues.set_index('Address'), on='Address')
venues_merged.shape # check the last columns!

NameError: name 'df_venues' is not defined

In [5]:
# venues_merged = pd.read_csv('venues_merged.csv')
# venues_merged.shape


In [6]:
regions = venues_merged['Region'].unique()
for region in regions:
    index = venues_merged[venues_merged['Region']==region]
    print(region)
    print(index['Cluster Labels'].unique())


London UK
[0]
Brisbane
[0 3 2 1]
Tokyo Japan
[0 3]
Christchurch New Zealand
[3 1 0 2]
Mumbai City India
[0 3 2 1]
Sydney
[3 0]
New York City US
[0 1 2 3]
New Delhi India
[0 3 2 1 4]
Beijing China
[3 0]
Los Angeles US
[0 1 3]
Auckland City New Zealand
[3 0 2 1]
Hanoi Vietnam
[0 3]
Kathmandu Nepal
[0 3]
Kuala Lumpur Malaysia
[0 3 2]
Shanghai China
[0 1]
Melbourne
[3 0]


In [17]:
kclusters = 5
def clusterMap(address, vLatitude_col, vLongitude_col, address_col, clusterLabel_col, zoom_start):

    geolocator = Nominatim(user_agent="explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

    # create map
    map_clusters = folium.Map(location=[latitude, longitude], zoom_start=zoom_start)

    # set color scheme for the clusters
    x = np.arange(kclusters)
    ys = [i + x + (i*x)**2 for i in range(kclusters)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map
    markers_colors = []
    for lat, lon, poi, cluster in zip(vLatitude_col, vLongitude_col, address_col, clusterLabel_col):
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
        
    return(map_clusters)

In [18]:
for region in regions:
    index = venues_merged[venues_merged['Region']==region]
    clusterMap(region, index['Venue Latitude'], index['Venue Longitude'], index['Address'], index['Cluster Labels'], 12)

The geograpical coordinate of London UK are 51.5073219, -0.1276474.
The geograpical coordinate of Brisbane are -27.4689682, 153.0234991.
The geograpical coordinate of Tokyo Japan are 55.7746222, 37.6326806.
The geograpical coordinate of Christchurch New Zealand are -43.530955, 172.6366455.
The geograpical coordinate of Mumbai City India are 18.9733536, 72.82810491917377.
The geograpical coordinate of Sydney are -33.8548157, 151.2164539.
The geograpical coordinate of New York City US are 40.7127281, -74.0060152.
The geograpical coordinate of New Delhi India are 28.6141793, 77.2022662.
The geograpical coordinate of Beijing China are 39.906217, 116.3912757.
The geograpical coordinate of Los Angeles US are 34.0536909, -118.2427666.
The geograpical coordinate of Auckland City New Zealand are -36.852095, 174.7631803.
The geograpical coordinate of Hanoi Vietnam are 21.0294498, 105.8544441.
The geograpical coordinate of Kathmandu Nepal are 27.708317, 85.3205817.
The geograpical coordinate of K